# Импорт

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from Classes.Regression import Regression
from Classes.RegressionTrainer import RegressionTrainer
from Functions.lossFunctions import euclideError, euclideErrorGradient
from Classes.Polinomizer import Polinomizer
from Functions.gradientDescentAlgorithm import gradientDescentAlgorithm
from Functions.errorFunctions import MAE, MAPE, MSE, RMSE, R2
from Functions.regulatorFinctions import Lasso, Ridge
# from sklearn.preprocessing import scale


# Загрузка датасета для регрессии.

In [2]:
models = pd.read_csv('../../Data/energy_task_moded.csv')
# scale(models)
models


,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,day,month,year,time
0,60.0,30.0,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,...,6.60,733.5,92.000000,7.000000,63.000000,5.3,11.0,1.0,2016.0,1020.0
1,60.0,30.0,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,...,6.48,733.6,92.000000,6.666667,59.166667,5.2,11.0,1.0,2016.0,1030.0
2,50.0,30.0,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,...,6.37,733.7,92.000000,6.333333,55.333333,5.1,11.0,1.0,2016.0,1040.0
3,50.0,40.0,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,...,6.25,733.8,92.000000,6.000000,51.500000,5.0,11.0,1.0,2016.0,1050.0
4,60.0,40.0,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,...,6.13,733.9,92.000000,5.666667,47.666667,4.9,11.0,1.0,2016.0,1060.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,100.0,0.0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,45.590000,...,22.70,755.2,55.666667,3.333333,23.666667,13.3,27.0,5.0,2016.0,1040.0
19731,90.0,0.0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,45.590000,...,22.60,755.2,56.000000,3.500000,24.500000,13.3,27.0,5.0,2016.0,1050.0
19732,270.0,10.0,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,45.730000,...,22.50,755.2,56.333333,3.666667,25.333333,13.3,27.0,5.0,2016.0,1060.0
19733,420.0,10.0,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,45.790000,...,22.30,755.2,56.666667,3.833333,26.166667,13.2,27.0,5.0,2016.0,1070.0


# Разбиваем на выборку

In [3]:
X_train, X_test, y_train, y_test = train_test_split(models.drop('Appliances', axis=1), models['Appliances'], test_size=0.1)


# Линейная регрессия

In [20]:
function_optimizer_error = lambda td, cd, w: gradientDescentAlgorithm(
    td=td, 
    cd=cd, 
    w=w, 
    lost_function=euclideError, 
    gradient=euclideErrorGradient, 
    nsteps=10, 
    merror=0.1, 
    step=0.1)


regressor = Regression()
trainer = RegressionTrainer(regressor, function_optimizer_error)
trainer.train(np.array(X_train, dtype=np.float64), np.array(y_train, dtype=np.float64))

y_predicted = regressor.predict(X_test)
# y_predicted
my_line_regression_classic_errors = {'MAE': MAE(y_predicted, y_test),
                                     'RMSE': RMSE(y_predicted, y_test),
                                     'MSE': MSE(y_predicted, y_test),
                                     'MAPE': MAPE(y_predicted, y_test),
                                     'R^2': R2(y_predicted, np.array(y_test))
                                    }

my_line_regression_classic_errors


{'MAE': 3.4385983185859963e+68,
 'RMSE': 3.4555510630502044e+68,
 'MSE': 1.1940833149347398e+137,
 'MAPE': 5.416915882412972e+66,
 'R^2': -1.0227777214915224e+135}

In [34]:
def function_optimizer_error(td, cd, w): return gradientDescentAlgorithm(
    td=td,
    cd=cd,
    w=w,
    lost_function=euclideError,
    gradient=euclideErrorGradient,
    nsteps=100,
    merror=0.01,
    step=0.000000000001)


regressor = Regression()
trainer = RegressionTrainer(regressor, function_optimizer_error)
trainer.train(np.array(X_train, dtype=np.float64),
              np.array(y_train, dtype=np.float64))

y_predicted = regressor.predict(X_test)
# y_predicted
my_line_regression_classic_errors = {'MAE': MAE(y_predicted, y_test),
                                     'RMSE': RMSE(y_predicted, y_test),
                                     'MSE': MSE(y_predicted, y_test),
                                     'MAPE': MAPE(y_predicted, y_test),
                                     'R^2': R2(y_predicted, np.array(y_test))
                                     }

my_line_regression_classic_errors


{'MAE': 2701.027146489771,
 'RMSE': 2718.7547445604177,
 'MSE': 7391627.361069783,
 'MAPE': 43.029760432585896,
 'R^2': -63311.09636642919}

In [44]:
alpha = 10
def function_optimizer_error(td, cd, w): return gradientDescentAlgorithm(
    td=td,
    cd=cd,
    w=w,
    lost_function=lambda td, cd, w: euclideError(td, cd, w) + Lasso(w, alpha),
    gradient=lambda td, cd, w: euclideErrorGradient(
        td, cd, w) + alpha * w.shape[0],
    nsteps=100,
    merror=0.01,
    step=0.000000000001)


regressor = Regression()
trainer = RegressionTrainer(regressor, function_optimizer_error)
trainer.train(np.array(X_train, dtype=np.float64),
              np.array(y_train, dtype=np.float64))

y_predicted = regressor.predict(X_test)
# y_predicted
my_line_regression_classic_errors = {'MAE': MAE(y_predicted, y_test),
                                     'RMSE': RMSE(y_predicted, y_test),
                                     'MSE': MSE(y_predicted, y_test),
                                     'MAPE': MAPE(y_predicted, y_test),
                                     'R^2': R2(y_predicted, np.array(y_test))
                                     }

my_line_regression_classic_errors


{'MAE': 2687.040517596407,
 'RMSE': 2697.958777139814,
 'MSE': 7278981.563145762,
 'MAPE': 43.00719098185108,
 'R^2': -62346.242313991315}

In [42]:
def function_optimizer_error(td, cd, w): return gradientDescentAlgorithm(
    td=td,
    cd=cd,
    w=w,
    lost_function=lambda td, cd, w: euclideError(td, cd, w) + Ridge(w, 100),
    gradient=lambda td, cd, w: euclideErrorGradient(td, cd, w),
    nsteps=100,
    merror=0.01,
    step=0.000000000001)


regressor = Regression()
trainer = RegressionTrainer(regressor, function_optimizer_error)
trainer.train(np.array(X_train, dtype=np.float64),
              np.array(y_train, dtype=np.float64))

y_predicted = regressor.predict(X_test)
# y_predicted
my_line_regression_classic_errors = {'MAE': MAE(y_predicted, y_test),
                                     'RMSE': RMSE(y_predicted, y_test),
                                     'MSE': MSE(y_predicted, y_test),
                                     'MAPE': MAPE(y_predicted, y_test),
                                     'R^2': R2(y_predicted, np.array(y_test))
                                     }

my_line_regression_classic_errors


{'MAE': 2742.5771108490885,
 'RMSE': 2755.1130853656364,
 'MSE': 7590648.113152956,
 'MAPE': 43.7971099473189,
 'R^2': -65015.784714379864}

In [52]:
def function_optimizer_error(td, cd, w): return gradientDescentAlgorithm(
    td=td,
    cd=cd,
    w=w,
    lost_function=lambda td, cd, w: euclideError(td, cd, w),
    gradient=lambda td, cd, w: euclideErrorGradient(td, cd, w),
    nsteps=100,
    merror=0.01,
    step=0.000000000000001)


regressor = Regression()
trainer = RegressionTrainer(regressor, function_optimizer_error)
trainer.train(Polinomizer(np.array(X_train, dtype=np.float64), 2).polinomize(),
              np.array(y_train, dtype=np.float64))

y_predicted = regressor.predict(Polinomizer(np.array(X_test, dtype=np.float64), 2).polinomize())
y_predicted
my_line_regression_classic_errors = {'MAE': MAE(y_predicted, y_test),
                                     'RMSE': RMSE(y_predicted, y_test),
                                     'MSE': MSE(y_predicted, y_test),
                                     'MAPE': MAPE(y_predicted, y_test),
                                     'R^2': R2(y_predicted, np.array(y_test))
                                     }

my_line_regression_classic_errors


{'MAE': 58724.743060742,
 'RMSE': 65865.99201496293,
 'MSE': 4338328904.11516,
 'MAPE': 1020.6438036374235,
 'R^2': -37159434.15946557}